### Things that need to be done
 
* Create the a test database that has:
  * base_data
  * accel_data
  
* Add on the following tables
  * bearing_capacity
  
* Add on the followinig tables
  * bluedrop config
  * bluedrop calibration data

* Trim the top of the bearing data plot (doesn't make a lot of sense but sure)
* 
### Options for formattig data
* Use a JSON for a quickoverview of the survey metadata
* Use a csv format to store information about each of the drops

In [ ]:
# Library installs
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Local loads
from lib.data_classes.pffpFolder import pffpDataFolder
from lib.signal_processing.signal_function import moving_average, find_deriv_change, find_drops
from lib.general_functions.global_constants import GRAVITY_CONST
from lib.general_functions.helper_function import convert_time_units


In [ ]:
# Data folder
folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops"

# PFFP sensor data
calibration_dir = r"calibration_factors\BlueDrop_Calibration_Factors.xlsx"

# PFFP tip information
tip_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\calibration_factors\BluedDrop_Cone_Values.xlsx"


In [ ]:
# TODO: Combine the pffp_id and the read_calibration_excel_sheet just have the objet call inside
name = "foo"
id = 9
test_folder = pffpDataFolder(folder_dir, pffp_id = id, calibration_factor_dir=calibration_dir, survey_name = name)

# Read the calibration data
test_folder.read_calibration_excel_sheet()
test_folder.calibration_excel_sheet

# Select the specific calibration params
if id == 3:
    test_folder.get_sensor_calibration_params("Sequim_2021")
elif id == 1:
    test_folder.get_sensor_calibration_params("March_2023")
elif id ==9:
    test_folder.get_sensor_calibration_params("October_2023")
else:
    raise ValueError("not a valid pffp id")

In [ ]:
# Init a pffpFile instances for each binary file and store the results in the folder
test_folder.store_pffp_files(recursive = False)

print(test_folder) # Information about the folder

In [ ]:
# Analyze the selected pffp files
test_folder.analyze_all_files(store_df=True, use_pore_pressure=True, 
                              select_accel=["2g_accel", "50g_accel", "250g_accel"],
                              debug = False)
print(test_folder)

test_folder.process_drop_files()

# Store the first drop file
file = test_folder.pffp_drop_files[0]
drop = file.drops[0]

pffp_id = test_folder.pffp_id
drop.get_pffp_tip_values(pffp_id=pffp_id, tip_type="cone", date_string = "April_2024", file_dir = tip_dir)

drop.water_drop = 1
drop.convert_tip_vals()



In [ ]:
accel = file.concat_accel
time = file.df["Time"] * 60
pressure = file.df["pore_pressure"]

In [ ]:
drop.quick_view_impulse()

In [ ]:
display(drop.bearing_dfs["mantle"])

In [ ]:
qsbc_vals = [0.2, 0.5, 0.9, 1.2, 1.5]

for name in ["mantle", "projected"]:
    drop.calc_drop_contact_area(area_type = name)
    drop.calc_drop_dynamic_bearing(area_type = name)
    for val in qsbc_vals:
        drop.calc_drop_qs_bearing(k_factor = val, area_type = name)
        drop.calc_drop_qs_bearing(k_factor = val, area_type = name)

In [ ]:
drop.bearing_dfs["projected"]

In [ ]:

displacement = drop.impulse_df["displacement"]
df = drop.bearing_dfs["projected"]

end = -17
for val in qsbc_vals:
    column = "qsbc_proj_{}".format(val)
    k_name=  "k = {}".format(val)
    bearing = df[column][:end]
    bearing = bearing/1e3
    plt.plot(bearing, displacement[:end] * 100, label= k_name)

plt.legend()
plt.gca().invert_yaxis()
plt.ylabel("Penetration Depth (cm)")
plt.xlabel("QSBC (kPa)")
plt.show()

In [ ]:
# Make the drop id
drop.make_drop_name()
drop.get_drop_datetime(file)
drop.get_peak_impulse_deceleration()

print(drop.name)

In [ ]:
import sqlite3

In [ ]:
from BlueDrop_db_lib.dbClass import PFFPDatabase

In [ ]:
# Input fake date information
survey_year = "24"
survey_month = "01"
survey_start_day = "01"

survey_name = test_folder.survey_name
survey_id = "{}_YR{}{}{}".format(survey_name, survey_year, survey_month, survey_start_day)

bluedrop_id = test_folder.pffp_id
pffp_calibration_df = test_folder.calibration_params
pffp_calibration_dict = {"pffp_id":bluedrop_id, "pffp_calibration_df":pffp_calibration_df}

location = "foo_location"
transect = "foo_transect"
latitude = 1.0
longitude = 2.0


In [ ]:
location_dict = {"survey_id":survey_id, "survey_name":survey_name, "location_name":location, "transect":transect, "latitude":latitude, "longitude":longitude}

In [ ]:
database = PFFPDatabase("foo3")
database.create_database()

df = drop.pffp_config["tip_props"]

In [ ]:
sub_df =list( df[df['Properties'] != 'radius_coeff']["units"])
display(sub_df)

In [ ]:
database.add_drop_data(drop, pffp_calibration_dict=pffp_calibration_dict, location_dict=location_dict, pffp_config_dict=drop.pffp_config)

In [ ]:
test_folder.calibration_params

In [ ]:
drop.pffp_config

In [ ]:
display(database.get_table_as_df(name = "drops"))

In [ ]:

display(database.get_table_as_df(name = "accel"))

In [ ]:
display(database.get_table_as_df(name = "bearing"))

In [ ]:
df=  database.get_table_as_df(name = "pffp_calib")
display(df)

In [ ]:
df=  database.get_table_as_df(name = "survey")
display(df)

In [ ]:
display(database.get_table_as_df(name = "pffp_config"))

In [ ]:
database.close_database()